In [1]:
from datasets import load_dataset

dataset = load_dataset("jkhedri/psychology-dataset")
dataset = dataset["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /home/diwank/.cache/huggingface/datasets/jkhedri___parquet/jkhedri--psychology-dataset-2cd732f026dd7c06/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en")

In [11]:
def embed(rows, idx):
    questions = rows["question"]
    embeddings = list(model.encode(questions, normalize_embeddings=True))
    return dict(idx=idx, embedding=embeddings)

dataset = dataset.map(embed, batched=True, with_indices=True)

Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [12]:
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

all_embeddings = dataset["embedding"]
all_embeddings = np.array(all_embeddings)

sim_matrix = 1 - pairwise_distances(all_embeddings, metric="cosine")

In [50]:
results = np.argwhere(sim_matrix>=0.95).tolist()

In [52]:
top_right = [(x, y) for [x, y] in results if x < y]

In [54]:
to_exclude = set()

for key, val in top_right:
    if key in to_exclude:
        continue

    to_exclude.add(val)

to_keep = list( set(range(len(dataset))) - to_exclude )

In [55]:
dataset = dataset.select(to_keep)

In [56]:
len(dataset)

1162

In [57]:
def to_dialog(row):
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_random_exponential,
    )

    import openai
    openai.api_key = "XXX"

    INSTRUCTION = (
        "You are an expert psychologist and acclaimed cognitive behavior researcher."
        " Given below is an exchange between two people A and B."
        " A is expressing a concern about their mental health and B is giving them advice from their perspective."
        "\n\nAs an expert in the field, think about the exchange and then evaluate whether it's a good advice or not."
        " To back up your evaluation, lay down your reasons for why you think so in detail and add remarks for what B can do better."
    )


    from redis import StrictRedis
    from redis_cache import RedisCache

    client = StrictRedis(host="localhost", decode_responses=True)
    cache = RedisCache(redis_client=client)

    @cache.cache()
    @retry(wait=wait_random_exponential(min=1, max=90), stop=stop_after_attempt(100))
    def completion_with_backoff(sample):

        messages = [dict(
            role="user",
            content=INSTRUCTION + "\n\n" + sample
        )]
        
        completion = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0.25,
        )
        
        result = completion.choices[0].message["content"]
        return result

    j_evaluation = completion_with_backoff(
        f'A: {row["question"]}'
        f'\n\nB: {row["response_j"]}'
    )

    k_evaluation = completion_with_backoff(
        f'A: {row["question"]}'
        f'\n\nB: {row["response_k"]}'
    )
    
    row["j_evaluation"] = j_evaluation
    row["k_evaluation"] = k_evaluation
    
    return row

In [58]:
dataset = dataset.map(to_dialog, num_proc=20)

Map (num_proc=20):   0%|          | 0/1162 [00:00<?, ? examples/s]

In [61]:
dataset[0]["j_evaluation"], "", dataset[0]["k_evaluation"]

("The advice given by B is generally good. They acknowledge A's feelings, normalize the experience of anxiety, and express a willingness to help A understand and manage their anxiety. This approach is supportive and empathetic, which is crucial when dealing with mental health issues.\n\nB also suggests identifying triggers and developing coping strategies, which is a common and effective approach in cognitive behavioral therapy. This involves understanding what situations or thoughts might be causing the anxiety and then developing strategies to manage these triggers. This can help A to feel more in control of their anxiety and reduce its impact on their life.\n\nHowever, B could improve their advice by suggesting that A seek professional help. While it's great to offer support and help as a friend, anxiety can be a serious mental health issue that requires professional treatment. B should encourage A to speak to a mental health professional who can provide a proper diagnosis and treat

In [62]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

In [72]:
def to_chatml(row):
    question = row["question"]
    response_j = row["response_j"]
    response_k = row["response_k"]
    j_evaluation = row["j_evaluation"]
    k_evaluation = row["k_evaluation"]

    chatmls = [
        [
            situation(
                f"Samantha is observing an exchange between two people A and B."
                " A is expressing a concern about their mental health and B is giving them advice from their perspective."
                " As an expert psychology, she is evaluating the advice."
            ),
            person(question, name="Person A"),
            person(response_j, name="Person A"),
            thought("Evaluation of Person B's response: " + j_evaluation),
        ]
        for r, e in [
            (response_j, j_evaluation),
            (response_k, k_evaluation),
        ]
    ]

    return dict(chatmls=chatmls)

dataset = dataset.map(to_chatml)

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

In [73]:
def unbatch(rows):
    chatmlss = rows["chatmls"]
    chatml = [
        c
        for cs in chatmlss
        for c in cs
    ]

    return dict(chatml=chatml)

ds = dataset.map(unbatch, batched=True, remove_columns=list(set(dataset.column_names) - {"chatml"}))

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

In [74]:
ds.push_to_hub("diwank/samantha-psychology-data", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]